### This will be a capstone project

In [1]:
import pandas as pd
import numpy as np
import requests
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geocoder --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

!conda install --yes  beautifulsoup4


from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

source = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago').text

soup = BeautifulSoup(source, 'html5lib')

In [3]:
commArea = []
neighborhoodList = []

In [4]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        neighborhoodList.append(cells[0].text.rstrip('\n'))
        commArea.append(cells[1].text.rstrip('\n'))

In [5]:
chi_df = pd.DataFrame({"Neighborhood": neighborhoodList, "Community": commArea})

In [6]:
chi_df.head(5)

,Neighborhood,Community
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [7]:
chi_df.shape

(246, 2)

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chicago, Illinois'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in chi_df["Neighborhood"].tolist() ]

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [13]:
chi_df['Latitude'] = df_coords['Latitude']
chi_df['Longitude'] = df_coords['Longitude']

In [14]:
chi_df

,Neighborhood,Community,Latitude,Longitude
0,Albany Park,Albany Park,41.968290,-87.723380
1,Altgeld Gardens,Riverdale,41.654410,-87.602250
2,Andersonville,Edgewater,41.980460,-87.668340
3,Archer Heights,Archer Heights,41.811540,-87.725560
4,Armour Square,Armour Square,41.834580,-87.631890
5,Ashburn,Ashburn,41.747850,-87.709950
6,Ashburn Estates,Ashburn,41.941674,-88.198809
7,Auburn Gresham,Auburn Gresham,41.743190,-87.655040
8,Avalon Park,Avalon Park,41.745070,-87.588160
9,Avondale,Avondale,41.939250,-87.711250


In [15]:
address = 'Chicago, Illinois'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago, Illinois {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago, Illinois 41.8755616, -87.6244212.


In [16]:
# create map of New York using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(chi_df['Latitude'], chi_df['Longitude'],chi_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [61]:
CLIENT_ID = 'EI05AG1VREBECM5VMGUXL1UN0A4A5BROAR2P5S0B42UUGC5C' # your Foursquare ID
CLIENT_SECRET = 'JH351FIX5HRK5AG0S1IUDXRAAVN3CJECC3A2ABYN1G5H2OLM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 15
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
search_query = 'Russian'
radius = 20000
lat = 41.968290000000025
long = -87.72337999999996

Your credentails:
CLIENT_ID: EI05AG1VREBECM5VMGUXL1UN0A4A5BROAR2P5S0B42UUGC5C
CLIENT_SECRET:JH351FIX5HRK5AG0S1IUDXRAAVN3CJECC3A2ABYN1G5H2OLM


In [62]:
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
# url
# create the API request URL
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&query={},&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        search_query,
        lat,
        long,
        radius,
        LIMIT)
    
results = requests.get(url).json()['response']



In [63]:
results


{'suggestedFilters': {'header': 'Tap to show:',
  'filters': [{'name': '$-$$$$', 'key': 'price'},
   {'name': 'Open now', 'key': 'openNow'}]},
 'headerLocation': 'Chicago',
 'headerFullLocation': 'Chicago',
 'headerLocationGranularity': 'city',
 'query': 'russian',
 'totalResults': 21,
 'suggestedBounds': {'ne': {'lat': 42.1482901800002, 'lng': -87.4817384865017},
  'sw': {'lat': 41.78828981999985, 'lng': -87.96502151349823}},
 'groups': [{'type': 'Recommended Places',
   'name': 'recommended',
   'items': [{'reasons': {'count': 0,
      'items': [{'summary': 'This spot is popular',
        'type': 'general',
        'reasonName': 'globalInteractionReason'}]},
     'venue': {'id': '4b64e8e7f964a52031d92ae3',
      'name': 'Jibek Jolu',
      'location': {'address': '5047 N Lincoln Ave',
       'crossStreet': 'at W Winnemac Ave',
       'lat': 41.97310738566933,
       'lng': -87.6914670054359,
       'labeledLatLngs': [{'label': 'display',
         'lat': 41.97310738566933,
         'l

In [18]:

venues = []


In [23]:

for lat, long, neighborhood in zip(chi_df['Latitude'], chi_df['Longitude'], chi_df['Neighborhood']):
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        latitude,
        longitude,
        search_query,
        radius,
        LIMIT)

    
    # make the GET request
    results = requests.get(url).json()['response']['venues']
    
#   results = requests.get(url).json()["response"]['groups'][0]['items']
    
# return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['name'], 
            venue['location']['lat'], 
            venue['location']['lng'],  
            venue['categories'][0]['name']))

KeyError: 'venues'

In [35]:
venues 



[('Albany Park',
  41.968290000000025,
  -87.72337999999996,
  'Russian Tea Time',
  41.879587803342204,
  -87.62505507506901,
  'Russian Restaurant'),
 ('Altgeld Gardens',
  41.65441000000004,
  -87.60224999999997,
  'Russian Tea Time',
  41.879587803342204,
  -87.62505507506901,
  'Russian Restaurant'),
 ('Andersonville',
  41.98046000000005,
  -87.66833999999994,
  'Russian Tea Time',
  41.879587803342204,
  -87.62505507506901,
  'Russian Restaurant'),
 ('Archer Heights',
  41.811540000000036,
  -87.72555999999997,
  'Russian Tea Time',
  41.879587803342204,
  -87.62505507506901,
  'Russian Restaurant'),
 ('Armour Square',
  41.834580000000074,
  -87.63188999999994,
  'Russian Tea Time',
  41.879587803342204,
  -87.62505507506901,
  'Russian Restaurant'),
 ('Ashburn',
  41.74785000000003,
  -87.70994999999994,
  'Russian Tea Time',
  41.879587803342204,
  -87.62505507506901,
  'Russian Restaurant'),
 ('Ashburn Estates',
  41.94167399999999,
  -88.19880887807226,
  'Russian Tea Time'

In [21]:
venues_df = pd.DataFrame(venues)


In [22]:
venues_df

,0,1,2,3,4,5,6
0,Albany Park,41.968290,-87.723380,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
1,Altgeld Gardens,41.654410,-87.602250,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
2,Andersonville,41.980460,-87.668340,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
3,Archer Heights,41.811540,-87.725560,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
4,Armour Square,41.834580,-87.631890,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
5,Ashburn,41.747850,-87.709950,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
6,Ashburn Estates,41.941674,-88.198809,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
7,Auburn Gresham,41.743190,-87.655040,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
8,Avalon Park,41.745070,-87.588160,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
9,Avondale,41.939250,-87.711250,Russian Tea Time,41.879588,-87.625055,Russian Restaurant
